# Data Engineering 1: Graded Lab 01
---------------

#### Grading
For this graded lab you can get a total of 10 points. These 10 points count 10% of your final grade for the course.

#### Start
Start of the Graded Lab 01 is **Thursday, March 21st at 23:55**.

#### Deadline
Deadline for the submission of the Graded Lab 01 is **Thursday, April 11th at 23:59**.

#### Note
Check each result carefully. Use data filter, cleaning, and transformation methods wherever needed. The data can sometimes be really messy and have hidden issues.

#### Submission
You are allowed to submit the solution in groups of **two or three** students.
Submit your GradedLab01.ipynb file renamed to FirstnameStudent01LastnameStudent01_FirstnameStudent02LastnameStudent02_FirstnameStudent03LastnameStudent03.ipynb in moodle.   
Please submit a runnable python jupyter notebook file.
All other submissions will be rejected and graded with 0 points.

##### Task 01: Structured Data [4 points].    
The imdb.csv file contains a dataset extracted from IMDB with several attributes. For example, the title, plot, and the language of the movies. Read the imdb.csv file in a pandas dataframe and try to answer the following questions.

##### __(a)  What is the min, max, and the average value of the attribute 'rating' for all movies? [0.5 points].__ 

In [1]:
import datetime
import math

import numpy as np
!pip install pandas spacy

In [2]:
import pandas as pd

data = pd.read_csv('imdb.csv')

data.head()

data = data.where(data.kind == 'movie').dropna()

print(f"Min: {data.rating.min()}")
print(f"Max: {data.rating.max()}")
print(f"Average: {data.rating.mean():.1f}")

Min: 8.1
Max: 9.3
Average: 8.4


##### __(b) What is the min, max, and the average value of the attribute 'cumulative worldwide gross' for all movies? [0.5 points].__ 

In [3]:
data["cumulative worldwide gross"] = data["cumulative worldwide gross"].str.replace(r'\s.+', '', regex=True).replace(r'\D', '', regex=True).astype(float)

print(f"Min: {data['cumulative worldwide gross'].min()}")
print(f"Max: {data['cumulative worldwide gross'].max()}")
print(f"Average: {data['cumulative worldwide gross'].mean():.1f}")

Min: 6540000.0
Max: 2797800564.0
Average: 390489624.7


##### __(c) What is the min, max, and the average value of the attribute 'cumulative worldwide gross' grouped by genre of the movies? [1 point].__ 

In [4]:
data['genres'] = data['genres'].apply(lambda x: x.replace("'", '').strip('][').split(', ') if not isinstance(x, list) else x)
data = data.explode('genres')
data.groupby(by=['genres'], dropna=True)['cumulative worldwide gross'].min()
data.groupby(by=['genres'], dropna=True)['cumulative worldwide gross'].max()
data.groupby(by=['genres'], dropna=True)['cumulative worldwide gross'].mean()

genres
Action       6.331150e+08
Adventure    6.941635e+08
Animation    6.597255e+08
Biography    2.393391e+08
Comedy       5.031645e+08
Crime        2.846189e+08
Drama        3.398912e+08
Family       6.597255e+08
Fantasy      5.482189e+08
History      2.548299e+08
Horror       2.245637e+08
Music        3.929283e+08
Musical      6.325005e+08
Mystery      2.488491e+08
Romance      2.931044e+08
Sci-Fi       6.745759e+08
Sport        1.553263e+08
Thriller     3.360929e+08
War          3.641424e+08
Western      3.362419e+08
Name: cumulative worldwide gross, dtype: float64

##### __(d) What is the profit (defined as cumulative worldwide gross - budget) of each movie? [1 point].__ 

In [5]:
data.budget = data.budget.str.replace(r'\D', '', regex=True).astype(float)
profit = data['cumulative worldwide gross'] - data.budget
profit

0      33500000.0
1     239066411.0
1     239066411.0
2     819558444.0
2     819558444.0
         ...     
94    399967620.0
95    191639112.0
96    402200000.0
96    402200000.0
96    402200000.0
Length: 299, dtype: float64

##### __(e)  What is the min, max, and the average value of the atttribute 'opening weekend united states' per month? [1 point].__ 

In [6]:
import datetime

data['opening weekend united states']= data['opening weekend united states'].apply(lambda x: x.replace("$", '').split(', ') if not isinstance(x, list) else x)
data = data.merge(data['opening weekend united states'].apply(lambda s: pd.Series({'opening weekend united states income': float(s[0].replace(',', '')), 'opening weekend united states dates': datetime.datetime.strptime(s[1], '%d %b %Y')}) if len(s) > 1 else pd.Series({'opening weekend united states income': np.NaN, 'opening weekend united states dates': np.NaN})), left_index=True, right_index=True)

data.groupby(by=[data['opening weekend united states dates'].dt.month], dropna=True)['opening weekend united states income'].min()
data.groupby(by=[data['opening weekend united states dates'].dt.month], dropna=True)['opening weekend united states income'].max()
data.groupby(by=[data['opening weekend united states dates'].dt.month], dropna=True)['opening weekend united states income'].mean()

opening weekend united states dates
1     1.326144e+07
2     2.974682e+07
3     3.421716e+07
4     1.593046e+08
5     5.929679e+07
6     5.916076e+07
7     5.430706e+07
8     1.121913e+07
9     1.056989e+07
10    2.193238e+07
11    2.672944e+07
12    1.964329e+07
Name: opening weekend united states income, dtype: float64

#### Task 02: Unstructured Data [6 points]. 
This time use the files imdb.csv from Task 01 and the imdb2.csv file. In this task we mainly use the content of the attribute 'plot' for the implementation of a retrieval system.

##### __(a)  Read the files imdb.csv and imdb2.csv in a dataframe called imdb. Add a column which is called 'plotterms' to each item. The new column should contain all terms (lower-case and cleaned from special characters) from the plot attribute which are not contained in the stopwords_english.txt file. [0.5 points]__ 

In [7]:
!spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 495.5 kB/s eta 0:00:26
      --------------------------------------- 0.2/12.8 MB 1.5 MB/s eta 0:00:09
     -- ------------------------------------- 0.8/12.8 MB 4.4 MB/s eta 0:00:03
     ----- ---------------------------------- 1.6/12.8 MB 6.5 MB/s eta 0:00:02
     -------- ------------------------------- 2.6/12.8 MB 8.3 MB/s eta 0:00:02
     ----------- ---------------------------- 3.7/12.8 MB 10.0 MB/s eta 0:00:01
     --------------- ------------------------ 4.8/12.8 MB 11.5 MB/s eta 0:00:01
     -------------------- ------------------- 6.6/12.8 MB 13.6 MB/s eta 0:00:01
     ------------------------ --------------- 8.0/12.8 MB 15.0 MB/s eta 0:00:01
     ------------------------------ --------- 9.9/12.8 MB 17.1 MB/s eta 0:00:01
     ------------------------------------ -- 11.8/12.8 MB 27

In [3]:
import pandas as pd
import spacy
import re

nlp = spacy.load('en_core_web_sm')

data = pd.read_csv('imdb2.csv')
imdb = pd.read_csv('imdb.csv')

imdb = pd.concat([data, imdb])

with open('stopwords_english.txt', 'r+', encoding='utf-8') as file:
    stopwords = file.readlines()
    stopwords = [sopword.strip() for sopword in stopwords]

def tokenize(text):
    doc = nlp(text)
    return [token.lemma_.lower() for token in doc if token.lemma_.lower() not in stopwords]

imdb['plot'] = imdb['plot'].apply(lambda x: '. '.join([re.sub(r'[^\w\s]', '', y) for y in x.split(', ')]))
imdb['plotterms'] = imdb['plot'].apply(tokenize)
imdb

,id,kind,title,languages,genres,rating,director,cumulative worldwide gross,opening weekend united states,budget,plot,plotterms
0,120669,movie,Fear and Loathing in Las Vegas,['English'],"['Adventure', 'Comedy', 'Drama']",7.5,Terry Gilliam,0,0,0,An oddball journalist and his psychopathic law...,"[oddball, journalist, psychopathic, lawyer, tr..."
1,1343092,movie,The Great Gatsby,['English'],"['Drama', 'Romance']",7.2,Baz Luhrmann,0,0,0,A writer and wall street trader. Nick. finds h...,"[writer, wall, street, trader, ., nick, ., dra..."
2,2069861,movie,My Favorite Movie,['English'],"['Comedy', 'Family']",8.4,Martin Rogers,0,0,0,Dave has the American Dream and hates it His e...,"[dave, american, dream, hate, eccentric, frien..."
3,2267368,movie,Joshua Tree,['English'],['Drama'],6.6,Li Cheng,0,0,0,Set in the US heartland following the 2008 eco...,"[set, heartland, follow, 2008, economic, colla..."
4,13560124,movie,After America,"['English', 'Somali']","['Drama', 'Fantasy', 'Mystery']",4.5,Jake Yuzna,0,0,0,A group of criminal justice deescalation worke...,"[group, criminal, justice, deescalation, worke..."
...,...,...,...,...,...,...,...,...,...,...,...,...
154,2085059,tv series,Black Mirror,['English'],"['Drama', 'Mystery', 'Sci-Fi', 'Thriller']",8.8,NaN,000000000,000000000,000000000,An anthology series exploring a twisted. hight...,"[anthology, series, explore, twist, ., hightec..."
155,2707408,tv series,Narcos,"['English', 'Spanish']","['Biography', 'Crime', 'Drama', 'Thriller']",8.8,NaN,000000000,000000000,000000000,A chronicled look at the criminal exploits of ...,"[chronicled, criminal, exploit, colombian, dru..."
156,98936,tv series,Twin Peaks,"['English', 'Icelandic', 'Afrikaans', 'Norwegi...","['Crime', 'Drama', 'Mystery', 'Thriller']",8.8,NaN,000000000,000000000,000000000,An idiosyncratic FBI agent investigates the mu...,"[idiosyncratic, fbi, agent, investigate, murde..."
157,4934214,tv series,Taskmaster,['English'],"['Comedy', 'Game-Show']",9.0,NaN,000000000,000000000,000000000,Five comedians are set tasks challenging their...,"[comedian, set, task, challenge, creativity, w..."


##### __(b)  Create an inverted index for the terms in the 'plotterms' column. Use a datastructure of your choice for the implementation [0.5 points]__ 

In [4]:
index = {}

def append_index(item):
    term_counter = {}
    curr_doc = item.id
    for term in item.plotterms:
        if term_counter.get(term) is None:
            term_counter[term] = 1
        else:
            term_counter[term] += 1
    
    for term, count in term_counter.items():
        if index.get(term) is None:
            index[term] = [(curr_doc, count)]
        else:
            index[term].append((curr_doc, count))
    
imdb.apply(append_index, axis=1)
index

{'oddball': [(120669, 1)],
 'journalist': [(120669, 2), (1877514, 1), (12392504, 1)],
 'psychopathic': [(120669, 1), (477348, 1)],
 'lawyer': [(120669, 3), (2802850, 1), (3032476, 5)],
 'travel': [(120669, 1),
  (80684, 1),
  (816692, 1),
  (6966692, 2),
  (266543, 1),
  (2395695, 1),
  (2861424, 4),
  (52520, 2),
  (303461, 3),
  (2802850, 1),
  (2098220, 1)],
 'las': [(120669, 1), (373862, 3)],
 'vegas': [(120669, 2), (373862, 3)],
 'series': [(120669, 1),
  (110912, 1),
  (758758, 1),
  (903747, 2),
  (795176, 1),
  (185906, 2),
  (7366338, 2),
  (306414, 1),
  (417299, 1),
  (6769208, 1),
  (2395695, 1),
  (9253866, 1),
  (944947, 1),
  (2861424, 2),
  (71075, 2),
  (1475582, 1),
  (52520, 1),
  (1877514, 5),
  (103359, 2),
  (386676, 1),
  (1806234, 1),
  (296310, 1),
  (2356777, 1),
  (98904, 2),
  (81912, 2),
  (2098220, 1),
  (98769, 1),
  (92337, 1),
  (108778, 1),
  (7137906, 1),
  (4742876, 2),
  (63929, 1),
  (1508238, 1),
  (264235, 1),
  (2085059, 4),
  (4934214, 2),
  (2

##### __(c) Take the inverted index and create the posting list for the query 'american'. Return the posting list as well as the top5 items scored by the rating attribute. [0.5 points]__ 

In [5]:
def get_posting_list(query):
    doc = nlp(query)
    all_documents = []
    
    for token in doc:
        token = token.lemma_.lower()
        result = index.get(token)
        all_documents = list({*all_documents, *[i[0] for i in result]})

    return all_documents, imdb[imdb.id.isin(all_documents)].sort_values(by=['rating'], ascending=False).id.head(5).tolist()
    
get_posting_list('american')

([46912,
  5092484,
  2069861,
  373862,
  2267368,
  2365001,
  1877514,
  306414,
  381681,
  98769,
  1343092,
  112471,
  88763,
  13560124,
  120669],
 [306414, 1877514, 98769, 88763, 2069861])

##### __(d) Take the inverted index and create the posting lists for the query terms 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot). Your merging algorithm should be efficient and reduce the number of comparison to a minimum. Don't use existing python methods (like intersect or in) for the merge algorithm. Return the merged posting list as well as the top5 items scored by the rating attribute. [1 point]__ 

In [6]:
def intersection_postings(posting1, posting2):
    merged = []
    for i in posting1:
        for j in posting2:
            if i == j:
                merged.append(i)
    return merged, imdb[imdb.id.isin(merged)].sort_values(by=['rating'], ascending=False).id.head(5).tolist()

american = get_posting_list('american')
dream = get_posting_list('dream')
intersection_postings(american[0], dream[0])

([5092484, 2069861, 373862, 2267368, 2365001, 1343092, 13560124, 120669],
 [2069861, 120669, 1343092, 2267368, 2365001])

##### __(e) Extend your index to be able to rank the resulting items from Task 2d by the occurrence of search terms in the plotterms. Execute the query 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot) again and print the resulting items.  [1 point]__

In [7]:
def get_ranked_posting(query):
    doc = nlp(query)
    results = {}
    all_documents = []

    for token in doc:
        token = token.lemma_.lower()
        result = index.get(token)
        results[token] = result
        all_documents = list({*all_documents, *[i[0] for i in result]})
        
    ranking = {i: 0 for i in all_documents}
    # RP1 & RP2
    for token, documents in results.items():
        for docu in documents:
            ranking[docu[0]] += docu[1]
    ranking = dict(sorted(ranking.items(), key=lambda item: -item[1]))
    
    return list(ranking.keys())[:]

get_ranked_posting('american dream')

[120669,
 1343092,
 2069861,
 5092484,
 13560124,
 2365001,
 373862,
 2267368,
 108778,
 98769,
 1877514,
 2380307,
 3032476,
 405159,
 52520,
 2098220,
 1375666,
 88763,
 46912,
 382932,
 112471,
 306414,
 381681]

##### __(f) Use the TF-IDF score and the cosine similarity to execute the query 'american' and 'dream' (the term 'american' as well as the term 'dream' should be contained in the plot) against all plots in the collection. Print the results with the corresponding ranking score. Also execute the query 'american' or 'dream' (the term 'american' or the term 'dream' should be contained in the plot).  [1.5 points]__

In [66]:
!pip install scipy

     ---------------------------------------- 0.0/59.0 kB ? eta -:--:--
     ------------- ------------------------ 20.5/59.0 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 59.0/59.0 kB 786.1 kB/s eta 0:00:00
   ---------------------------------------- 0.0/42.2 MB ? eta -:--:--
   ---------------------------------------- 0.5/42.2 MB 16.5 MB/s eta 0:00:03
   - -------------------------------------- 1.6/42.2 MB 16.6 MB/s eta 0:00:03
   -- ------------------------------------- 2.7/42.2 MB 19.1 MB/s eta 0:00:03
   ---- ----------------------------------- 4.2/42.2 MB 22.7 MB/s eta 0:00:02
   ---- ----------------------------------- 4.8/42.2 MB 21.9 MB/s eta 0:00:02
   ------ --------------------------------- 6.4/42.2 MB 22.6 MB/s eta 0:00:02
   -------- ------------------------------- 9.1/42.2 MB 27.7 MB/s eta 0:00:02
   ------------ --------------------------- 13.1/42.2 MB 46.7 MB/s eta 0:00:01
   ----------------- ---------------------- 18.1/42.2 MB 93.0 MB/s eta 0:

In [90]:
from scipy import spatial
import math

def merge_postings(posting1, posting2):
    return [*posting1, *posting2], ()


def calculate_tf(term, document):
    docs = index.get(term)
    count = 0
    for doc in docs:
        if doc[0] == document:
            count = doc[1]
    return count / len(imdb.loc[imdb.id == document].plotterms)

def calculate_idf(term):
    docs = index.get(term)
    return math.log(len(list(imdb.index)) / len(docs))

def calculate_tf_idf(terms, document):
    result = 0
    for term in terms:
        result += calculate_tf(term, document) * calculate_idf(term)
    return result

def calculate_similarity(terms, document):
    result = 0
    all_terms = list(index.keys())
    
    
    term_vector = [0 for _ in range(len(list(index.keys())))]
    doc_vector = [0 for _ in range(len(list(index.keys())))]
    for term in terms:
        term_vector[all_terms.index(term)] += 1
    for terms in imdb.loc[imdb.id == document].plotterms:
        for term in terms:
            doc_vector[all_terms.index(term)] += 1
        
        result += spatial.distance.cosine(term_vector, doc_vector)
    
    return result

def get_posting(query):
    doc = nlp(query)
    results = {}
    term_documents = {}
    algorithm = None
    
    for token in doc:
        token = token.lemma_.lower()
        if token == 'or':
            algorithm = merge_postings
        elif token == 'and':
            algorithm = intersection_postings
        else:
            result = index.get(token)
            results[token] = result
            term_documents[token] = [i[0] for i in result]
    
    # Correctly combine 2 posting lists
    terms = list(results.keys())
    all_documents = algorithm([i[0] for i in results[terms[0]]], [i[0] for i in results[terms[1]]])[0]
    
    # TFIDF Ranking
    ranking = {i: calculate_tf_idf(terms, i) for i in all_documents}
    
    # Similarity
    ranking = {doc: value * calculate_similarity(terms, doc) for doc, value in ranking.items()}
    
    ranking = dict(sorted(ranking.items(), key=lambda item: -item[1]))
    
    return ranking # Could add max return values
            

print(get_posting('american and dream'))
print(get_posting('american or dream'))

{120669: 17.094445890121136, 1343092: 15.579225172034914, 2069861: 11.58854643613605, 2365001: 8.5277813029429, 373862: 8.404166321909317, 2267368: 7.848790422450662, 5092484: 7.832701886823236, 13560124: 7.748399204961346}
{120669: 17.094445890121136, 1343092: 15.579225172034914, 2069861: 11.58854643613605, 2365001: 8.5277813029429, 373862: 8.404166321909317, 2267368: 7.848790422450662, 5092484: 7.832701886823236, 13560124: 7.748399204961346, 108778: 6.9116452532281505, 98769: 4.6904680419599805, 1877514: 2.3836770984241786, 381681: 2.361334694707772, 306414: 2.340662603638859, 46912: 2.3267926570082795, 2098220: 2.32036882423652, 3032476: 2.3098768046527987, 52520: 2.293451474961393, 112471: 2.2869616913853608, 1375666: 2.2732302378018736, 405159: 2.250442413841651, 88763: 2.24121380918683, 382932: 2.2398803263371168, 2380307: 2.20671837983322}
